In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xlrd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
settlements = pd.read_excel("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Settlements.xlsx", na_values="#NULL!")

In [4]:
fundamentals = pd.read_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fundamentals_Full.csv", na_values=' ')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,25,29,916,917,919,920,926,935,937,940,941,942,943,945,955,956,959,962,965,970,978,981,984,989,992,993,997,998,1008,1009,1014,1015,1016,1017,1018,1019,1020,1021,1024,1025,1028,1029,1031,1033,1034,1035,1036,1040,1041,1042,1052,1053,1054,1055,1056,1057,1058,1059,1066,1071,1072,1074,1075,1076,1077,1081,1084,1091,1094,1096,1098,1101,1102,1103,1105,1106,1107,1108,1116,1117,1119,1120,1121,1122,1123,1124,1125,1735,1736,1737,1742,1756,1757,1762) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
def excel_to_dictionary(sheetname, filename="/content/drive/MyDrive/Fiverr/PROJECT_MIMI/DataDictionary.xlsx"):
    abbreviation_dict = dict()
    
    abbreviation = xlrd.open_workbook(os.path.abspath(filename))
    abbreviation_sheet = abbreviation.sheet_by_name(sheetname)
    
    for i in range(1, abbreviation_sheet.nrows):
        abb = abbreviation_sheet.cell(i, 0).value
        abb_definition = abbreviation_sheet.cell(i, 1).value
        abbreviation_dict[abb] = abb_definition
        
    return abbreviation_dict

In [6]:
fundamentals_dict = excel_to_dictionary(sheetname="Fundamentals")

In [7]:
def rename_columns(df, df_dict):
  df.rename(columns=df_dict, inplace=True)
  df.columns = [column.lower().replace(" ", "_") for column in df.columns]
  return df

In [8]:
fundamentals = rename_columns(fundamentals, fundamentals_dict)
settlements.columns = [column.lower().replace(" ", "_") for column in settlements.columns]
settlements.rename(columns={"ticker": "ticker_symbol"}, inplace=True)

In [9]:
settlements.head()

,filingname,exchange,ticker_symbol,dismissed,settlementamount,filingyear
0,"AAC Holdings, Inc.",New York SE,AAC,Ongoing,NaN,2015
1,"Airtran Holdings, Inc.",New York SE,AAI,Yes,NaN,2011
2,Altisource Asset Management Corporation,New York SE,AAMC,Ongoing,NaN,2015
3,"APPLE Computer, Inc.",NASDAQ,AAPL,No,16500000.0,2006
4,"Avalanche Biotechnologies, Inc.",NASDAQ,AAVL,Ongoing,NaN,2015


In [10]:
settlements = settlements.sort_values(by=["ticker_symbol", "settlementamount", "filingyear"], ascending=[True, False, False])

In [11]:
fundamentals["sued"] = np.where(fundamentals["ticker_symbol"].isin(list(set(sorted(fundamentals["ticker_symbol"].unique())) & set(sorted(settlements["ticker_symbol"].unique())))), "yes", "no")
fundamentals["sued"].unique()

array(['no', 'yes'], dtype=object)

In [12]:
settlements[settlements["ticker_symbol"] == "AMTD"]

,filingname,exchange,ticker_symbol,dismissed,settlementamount,filingyear
86,TD Ameritrade Holding Corporation,New York SE,AMTD,Ongoing,NaN,2014
87,TD Ameritrade Holding Corporation,New York SE,AMTD,Ongoing,NaN,2014
85,TD Ameritrade Holding Corporation : Auction Ra...,NASDAQ,AMTD,Yes,NaN,2008


In [13]:
settlements = settlements.drop_duplicates(subset=["ticker_symbol"], keep="first")
settlements = settlements[["ticker_symbol", "settlementamount"]]

In [14]:
fund_settlement = pd.merge(fundamentals, settlements, how="left", on="ticker_symbol")

In [15]:
fund_settlement.shape

(61429, 1770)

In [16]:
fund_settlement.to_csv("/content/drive/MyDrive/Fiverr/PROJECT_MIMI/Fund_Settlement.csv", index=False)

In [17]:
settlements[settlements["ticker_symbol"] == "AAN"]

,ticker_symbol,settlementamount


In [18]:
fund_settlement.shape

(61429, 1770)